In [99]:
from pymongo import MongoClient
import pymongo
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import datetime
from bokeh.io import output_notebook 
from bokeh.plotting import figure, show, output_file, save
from bokeh.transform import cumsum
from bokeh.models import ColumnDataSource, LabelSet, TapTool, Tabs, Slider
from bokeh.layouts import row, column, layout
import pandas as pd

In [100]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [101]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classdb', 'config', 'epa', 'fruitsdb', 'local', 'met', 'petsitly_marketing', 'recall_data_for_sql', 'travel_db']


In [102]:
# assign the database to a variable name
db = mongo['recall_data_for_sql']

In [103]:
# review the collections in our new database
print(db.list_collection_names())

['recall_list']


In [104]:
# review a document in the customer_list collection
print(db.recall_list.find_one())

{'_id': ObjectId('65fa24d62008f898be8adb66'), 'Event_ID': 72241, 'Recalling_Firm': 'Kalman Health & Wellness, Inc. dba Essential Wellness Pharma', 'Substance_Name': "['na']", 'Product_Description': 'Progesterone 100 mg/mL in Corn Oil Injection, 2 mL vials, Rx only, Essential Wellness PHARMACY, 4625 N. University, Peoria, IL 61614.', 'Ingestion_Route': 'na', 'Reason': 'Lack of Assurance of Sterility', 'Classification': 'Class II', 'Quantity': '1 vial', 'Voluntary_Mandated': 'Voluntary: Firm initiated', 'Distribution': 'Nationwide', 'Initial_Notification_Method': 'Letter', 'Recall_Begins': '9/3/2015', 'Recall_End': '12/29/2017', 'Status': 'Terminated', 'City': 'Peoria', 'State': 'IL', 'Postal_Code': '61614-5828', 'Country': 'United States'}


In [105]:
# assign the collection to a variable
recall_list = db['recall_list']

In [106]:
#creating a Pandas dataframe to work with Bokeh
recall_data = pd.DataFrame(list(recall_list.find()))
recall_data.head()

,_id,Event_ID,Recalling_Firm,Substance_Name,Product_Description,Ingestion_Route,Reason,Classification,Quantity,Voluntary_Mandated,Distribution,Initial_Notification_Method,Recall_Begins,Recall_End,Status,City,State,Postal_Code,Country
0,65fa24d62008f898be8adb66,72241,"Kalman Health & Wellness, Inc. dba Essential W...",['na'],"Progesterone 100 mg/mL in Corn Oil Injection, ...",na,Lack of Assurance of Sterility,Class II,1 vial,Voluntary: Firm initiated,Nationwide,Letter,9/3/2015,12/29/2017,Terminated,Peoria,IL,61614-5828,United States
1,65fa24d62008f898be8adb67,86162,Albek De Mexico S.A. De C.V.,['na'],Assured Instant Hand Sanitizer Aloe & Moisturi...,na,CGMP Deviations,Class II,"27,000 bottles",Voluntary: Firm initiated,USA Nationwide,Press Release,7/30/2020,,Ongoing,San Juan Del Rio,,,Mexico
2,65fa24d62008f898be8adb68,85712,Teva Pharmaceuticals USA,"['AMPHETAMINE ASPARTATE MONOHYDRATE', 'AMPHETA...","Dextroamphetamine Saccharate, Amphetamine Aspa...",ORAL,Some bottles may contain mixed strengths of th...,Class II,"33,280 bottles",Voluntary: Firm initiated,Nationwide within the United State and Puerto ...,Letter,5/22/2020,5/25/2021,Terminated,Parsippany,NJ,07054-1120,United States
3,65fa24d62008f898be8adb69,74057,"Pharmedium Services, LLC",['na'],2 mcg/mL Fentanyl Citrate and 0.16% Bupivacain...,na,Presence of Particulate Matter,Class II,940 bags,Voluntary: Firm initiated,Nationwide,E-Mail,5/5/2016,3/6/2018,Terminated,Lake Forest,IL,60045-2506,United States
4,65fa24d62008f898be8adb6a,88905,Perrigo Company PLC,['OXYMETAZOLINE HYDROCHLORIDE'],"No Drip Nasal Spray, Oxymetazoline HCl 0.05% N...",NASAL,CGMP Deviations,Class II,"1,560 bottles",Voluntary: Firm initiated,Nationwide in the USA,Letter,10/26/2021,9/29/2023,Terminated,Allegan,MI,49010-9070,United States


In [107]:
# finding the Dtype of each column
recall_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   _id                          500 non-null    object
 1   Event_ID                     500 non-null    int64 
 2   Recalling_Firm               500 non-null    object
 3   Substance_Name               500 non-null    object
 4   Product_Description          500 non-null    object
 5   Ingestion_Route              500 non-null    object
 6   Reason                       500 non-null    object
 7   Classification               500 non-null    object
 8   Quantity                     500 non-null    object
 9   Voluntary_Mandated           500 non-null    object
 10  Distribution                 500 non-null    object
 11  Initial_Notification_Method  500 non-null    object
 12  Recall_Begins                500 non-null    object
 13  Recall_End                   500 no

In [108]:
# Converting Columns to datetime and creating new Recall_Start_Year  Column
recall_data['Recall_Begins'] = pd.to_datetime(recall_data['Recall_Begins'])
recall_data['Recall_End'] = pd.to_datetime(recall_data['Recall_End'])
recall_data['Recall_Start_Year'] = recall_data['Recall_Begins'].dt.year
recall_data.head()

,_id,Event_ID,Recalling_Firm,Substance_Name,Product_Description,Ingestion_Route,Reason,Classification,Quantity,Voluntary_Mandated,Distribution,Initial_Notification_Method,Recall_Begins,Recall_End,Status,City,State,Postal_Code,Country,Recall_Start_Year
0,65fa24d62008f898be8adb66,72241,"Kalman Health & Wellness, Inc. dba Essential W...",['na'],"Progesterone 100 mg/mL in Corn Oil Injection, ...",na,Lack of Assurance of Sterility,Class II,1 vial,Voluntary: Firm initiated,Nationwide,Letter,2015-09-03,2017-12-29,Terminated,Peoria,IL,61614-5828,United States,2015
1,65fa24d62008f898be8adb67,86162,Albek De Mexico S.A. De C.V.,['na'],Assured Instant Hand Sanitizer Aloe & Moisturi...,na,CGMP Deviations,Class II,"27,000 bottles",Voluntary: Firm initiated,USA Nationwide,Press Release,2020-07-30,NaT,Ongoing,San Juan Del Rio,,,Mexico,2020
2,65fa24d62008f898be8adb68,85712,Teva Pharmaceuticals USA,"['AMPHETAMINE ASPARTATE MONOHYDRATE', 'AMPHETA...","Dextroamphetamine Saccharate, Amphetamine Aspa...",ORAL,Some bottles may contain mixed strengths of th...,Class II,"33,280 bottles",Voluntary: Firm initiated,Nationwide within the United State and Puerto ...,Letter,2020-05-22,2021-05-25,Terminated,Parsippany,NJ,07054-1120,United States,2020
3,65fa24d62008f898be8adb69,74057,"Pharmedium Services, LLC",['na'],2 mcg/mL Fentanyl Citrate and 0.16% Bupivacain...,na,Presence of Particulate Matter,Class II,940 bags,Voluntary: Firm initiated,Nationwide,E-Mail,2016-05-05,2018-03-06,Terminated,Lake Forest,IL,60045-2506,United States,2016
4,65fa24d62008f898be8adb6a,88905,Perrigo Company PLC,['OXYMETAZOLINE HYDROCHLORIDE'],"No Drip Nasal Spray, Oxymetazoline HCl 0.05% N...",NASAL,CGMP Deviations,Class II,"1,560 bottles",Voluntary: Firm initiated,Nationwide in the USA,Letter,2021-10-26,2023-09-29,Terminated,Allegan,MI,49010-9070,United States,2021


In [109]:
#Recalls by Year Bokeh Graph
recall_totals = recall_data.groupby('Recall_Start_Year')[['Event_ID']].count()
source = ColumnDataSource(recall_totals)
recalls_by_year= figure(width = 300, height = 400, title = "Recall Counts by Year")
recalls_by_year.line(x = 'Recall_Start_Year', y = 'Event_ID', line_width = 2, source = source)
show(recalls_by_year)

In [119]:
# Grouping the data by ‘Country’ and counting the occurrences of ‘Event_ID’
recall_country = recall_data.groupby('Country')['Event_ID'].count().reset_index()
# Creating a ColumnDataSource with the grouped data
source = ColumnDataSource(recall_country)
# Creating the figure
reasons_by_country_plot = figure(title="Recall Counts by Country",
                                 height=300,
                                 width=800,  # Increased width for better visualization
                                 x_range=recall_country['Country'])  # Setting x_range to ensure all countries are shown
# Plotting the bar chart
reasons_by_country_plot.vbar(x='Country', top='Event_ID', source=source, width=0.5)
# Setting the orientation of the x-axis labels to vertical for better readability
reasons_by_country_plot.xaxis.major_label_orientation = 'vertical'
# Showing the plot
show(reasons_by_country_plot)